# Recognizing Flatlanders

## Data preprocessing

In [1]:
import h5py
from keras import backend as K
from keras.utils import np_utils
import numpy as np

Using TensorFlow backend.


Read the training and test data from the HDF5 files.

In [2]:
with h5py.File('train_data.h5', 'r') as h5_file:
    x_train = np.array(h5_file['x_values'])
    y_train = np.array(h5_file['y_values'])
with h5py.File('test_data.h5', 'r') as h5_file:
    x_test = np.array(h5_file['x_values'])
    y_test = np.array(h5_file['y_values'])

Determine whether the backend expects the channel to come first or last in the input shape.

In [3]:
img_channels, img_rows, img_cols = 1, x_train.shape[1], x_train.shape[2]

if K.image_data_format() == 'channels_first':
    shape_ord = (img_channels, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, img_channels)

Reshape the data accordingly.

In [4]:
x_train = x_train.reshape((x_train.shape[0], ) + shape_ord)
x_test = x_test.reshape((x_test.shape[0], ) + shape_ord)

Convert the input data to single precision floating point, and scale between 0 and 1.

In [5]:
x_train = x_train.astype(np.float32)/255.0
x_test = x_test.astype(np.float32)/255.0

Verify input data format and shape.

In [6]:
print(x_train.shape, x_train.dtype, x_train.min(), x_train.max())
print(x_test.shape, x_test.dtype, x_test.min(), x_test.max())

(1000, 100, 100, 1) float32 0.0 1.0
(1000, 100, 100, 1) float32 0.0 1.0


Convert output to one-hot encoding.

In [7]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

Verify output data format and shape.

In [8]:
print(y_train.shape, y_train.dtype, y_train.min(), y_train.max())
print(y_test.shape, y_test.dtype, y_test.min(), y_test.max())

(1000, 3) float32 0.0 1.0
(1000, 3) float32 0.0 1.0


## Model definition

In [16]:
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, LeakyReLU
from keras.layers.pooling import MaxPool2D
from keras.models import Sequential
from keras.optimizers import SGD, Adam

In [19]:
nr_filters = 32
nr_classes = y_train.shape[-1]
conv_x_size, conv_y_size = 5, 5
model = Sequential()
model.add(Conv2D(nr_filters, (conv_x_size, conv_y_size), strides=2,
                 padding='valid', input_shape=shape_ord))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.4))
model.add(Conv2D(nr_filters*2, (conv_x_size, conv_y_size), strides=2,
                 padding='valid'))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.4))
model.add(Conv2D(nr_filters*4, (conv_x_size, conv_y_size), strides=2,
                 padding='valid'))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(nr_classes))
model.add(Activation('softmax'))

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 48, 48, 32)        832       
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 22, 22, 64)        51264     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 9, 9, 128)         204928    
__________

In [21]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(),
              metrics=['accuracy'])

In [25]:
model.fit(x_train, y_train, epochs=1, batch_size=32,
         validation_data=(x_test, y_test))

Train on 1000 samples, validate on 1000 samples
Epoch 1/1
1000/1000 [==============================] - 38s 38ms/step - loss: 0.9377 - acc: 0.5240 - val_loss: 0.9720 - val_acc: 0.4660
